In [1]:
# Import packages

import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [2]:
# Configure SQL

def execute_statement(sql: str):
    with psycopg2.connect(host="localhost", database="thefantasybot", user="tbakely") as conn:
         df = pd.read_sql(sql, conn)
         return df

weekly_sql = """
select
	wd.player_id,
	wd.player_name,
	position,
	recent_team,
	wd.season,
	wd.week,
	carries,
	rushing_yards,
	rushing_tds,
	rushing_fumbles,
	rushing_fumbles_lost,
	rushing_first_downs,
	rushing_epa,
	--efficiency,
	--percent_attempts_gte_eight_defenders,
	--avg_time_to_los,
	--rush_yards_over_expected,
	--avg_rush_yards,
	--rush_yards_over_expected_per_att,
	--rush_pct_over_expected,
	wd.receptions,
	wd.targets,
	receiving_yards,
	receiving_tds,
	receiving_fumbles,
	receiving_fumbles_lost,
	receiving_air_yards,
	receiving_yards_after_catch,
	receiving_first_downs,
	receiving_epa,
	racr,
	target_share,
	air_yards_share,
	wopr,
	offense_snaps,
	offense_pct,
    redzone.redzone,
	(carries + wd.targets) as total_usage,
    wd.fantasy_points,
    wd.fantasy_points_ppr,
	roof,
	surface,
	weather_hazards,
	temp,
	humidity,
	wind_speed
from archive_data.weekly_data wd
left join archive_data.offense_snap_counts os
on wd.player_id = os.id
and wd.season = os.season
and wd.week = os.week
left join archive_data.ngs_rushing_data ngsr
on wd.player_id = ngsr.player_gsis_id
and wd.season = ngsr.season
and wd.week = ngsr.week
left join archive_data.ngs_receiving_data ngsrr
on wd.player_id = ngsrr.player_gsis_id
and wd.season = ngsrr.season
and wd.week = ngsrr.week
left join (select distinct rusher_player_id, game_id, season, week from archive_data.full_pbp) game_id
on wd.player_id = game_id.rusher_player_id
and wd.season = game_id.season
and wd.week = game_id.week
left join archive_data.game_data
on game_data.game_id = game_id.game_id
left join archive_data.redzone_snaps redzone
on wd.player_id = redzone.player_id
and wd.season = redzone.season
and wd.week = redzone.week
where position in ('WR', 'RB', 'TE')
and wd.season between 2016 and 2022;
"""

# Load weekly data from 2016-2022; Modify above query as needed
weekly = execute_statement(weekly_sql)

# Dealing with null values
weekly1 = weekly.copy()
weekly1.dropna(subset=["player_name"], inplace=True)
weekly1.dropna(subset=["offense_snaps"], inplace=True)

fill_na_cols = [
    "rushing_epa",
    "receiving_epa",
    "racr",
    "target_share",
    "air_yards_share",
    "wopr",
    "redzone",
]

weather_cols = [
    "roof",
    "surface",
    "weather_hazards",
    "temp",
    "humidity",
    "wind_speed",
]

for col in fill_na_cols:
    weekly1[col] = weekly[col].fillna(0)

weekly1 = weekly1[[col for col in weekly1.columns if col not in weather_cols]]
weekly1["week"] = weekly1["week"].astype(str)
weekly1["season"] = weekly1["season"].astype(str)

weekly1["scored"] = np.where((weekly1["receiving_tds"] > 0) | (weekly1["rushing_tds"] > 0), 1, 0)
weekly1["multi_score"] = np.where(weekly1["receiving_tds"] + weekly1["rushing_tds"] > 1, 1, 0)
weekly1["total_yards"] = weekly1["rushing_yards"] + weekly1["receiving_yards"]
weekly1["total_epa"] = weekly1["rushing_epa"] + weekly1["receiving_epa"]
weekly1["total_first_downs"] = weekly1["rushing_first_downs"] + weekly1["receiving_first_downs"]


try_columns = ["total_epa", "total_first_downs", "target_share", "redzone", "offense_pct", "total_usage"]


model_data = weekly1[try_columns]
model_data.head()

,total_epa,total_first_downs,target_share,redzone,offense_pct,total_usage
1,-0.258469,0.0,0.111111,0.0,0.66,4
2,0.303061,1.0,0.090909,0.0,0.66,3
3,0.905386,1.0,0.060000,0.0,0.40,3
6,0.298625,3.0,0.325581,2.0,1.00,14
7,2.561272,1.0,0.236842,1.0,1.00,9


In [3]:
from sklearn.model_selection import train_test_split

# Model set up
X = model_data.values
y = weekly1[["fantasy_points_ppr"]].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [26]:
# PyTorch Imports
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")


In [24]:
%%writefile data_preparation/v0.py

device = ("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train).float().to(device)
y_train_tensor = torch.tensor(y_train).float().to(device)

Overwriting data_preparation/v0.py


In [25]:
%run -i data_preparation/v0.py

In [49]:
%%writefile model_configuration/v0.py

# Set device
device = ("cuda" if torch.cuda.is_available() else "cpu")

# Set learning rate
lr = 0.001

# Create model and send to device
model = nn.Sequential(nn.Linear(6,1)).to(device)

# Define an SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define the MSE loss function
loss_fn = nn.MSELoss(reduction="mean")

Overwriting model_configuration/v0.py


In [50]:
%run -i model_configuration/v0.py

In [51]:
%%writefile model_training/v0.py

# Define number of epochs
n_epochs = 1000

for epoch in range(n_epochs):
    model.train()

    # Step 1 - Compute the output
    yhat = model(X_train_tensor)

    # Step 2 - Compute the loss
    loss = loss_fn(yhat, y_train_tensor)

    # Step 3 - Compute the gradients for b and w
    loss.backward()

    # Step 4 - Update parameters
    optimizer.step()
    optimizer.zero_grad()

Overwriting model_training/v0.py


In [52]:
%run -i model_training/v0.py

In [53]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[0.8105, 1.3459, 0.7200, 0.2973, 0.7444, 0.4834]])), ('0.bias', tensor([1.0452]))])


In [54]:
# Helper higher order function for training

def make_train_step_fn(model, loss_fn, optimizer):
    def perform_train_step_fn(X, y):
        model.train()

        # Step 1 - Compute the output
        yhat = model(X)

        # Step 2 - Compute the loss
        loss = loss_fn(yhat, y)

        # Step 3 - Compute the gradients for b and w
        loss.backward()

        # Step 4 - Update parameters
        optimizer.step()
        optimizer.zero_grad()

        # Return the loss
        return loss.item()
    
    return perform_train_step_fn

In [55]:
%run -i data_preparation/v0.py

In [56]:
%%writefile model_configuration/v1.py

device = ("cuda" if torch.cuda.is_available() else "cpu")

# Set learning rate
lr = 0.001

torch.manual_seed(0)

# Define model
model = nn.Sequential(nn.Linear(6,1)).to(device)

# Define SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define the MSE loss function
loss_fn = nn.MSELoss(reduction="mean")

# Create the train_step function for our model, loss, and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)


Writing model_configuration/v1.py


In [57]:
%run -i model_configuration/v1.py

In [58]:
%%writefile model_training/v1.py

# Define number of epochs
n_epochs = 1000

losses = []

for epoch in range(n_epochs):
    loss = train_step_fn(X_train_tensor, y_train_tensor)
    losses.append(loss)

Writing model_training/v1.py


In [61]:
%run -i model_training/v1.py

In [63]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[0.7875, 1.4565, 0.0289, 0.2645, 0.7500, 0.4758]])), ('0.bias', tensor([0.9314]))])


In [64]:
# What a custom class may look like
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

In [65]:
x_train_tensor = torch.as_tensor(X_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([-1.4441,  3.0000,  0.4400,  0.0000,  0.9400, 11.0000]), tensor([14.2000]))


In [66]:
# If we only have a couple of tensors, we can use TensorDataset

train_data = TensorDataset(X_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([-1.4441,  3.0000,  0.4400,  0.0000,  0.9400, 11.0000]), tensor([14.2000]))


In [68]:
# We now use PyTorch's DataLoader which we NEED to do batch gradient descent
# ALWAYS set shuffle=True for the training set to avoid data leakage, unless
# doing something like time series
# No need to shuffle the test sets because we are NOT computing their gradients

# Mini batch size: powers of 2 (e.g. 16, 32, 64, 128)

train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)

In [69]:
%%writefile data_preparation/v1.py

X_train_tensor = torch.as_tensor(X_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = TensorDataset(X_train_tensor, y_train_tensor)

train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)

Writing data_preparation/v1.py


In [70]:
%run -i data_preparation/v1.py

In [71]:
%run -i model_configuration/v1.py

In [72]:
%%writefile model_training/v2.py

# Define epochs
n_epochs = 1000

losses = []

for epoch in range(n_epochs):
    # mini batch loop
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        # Send each mini batch to device
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = train_step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    
    # average loss over all mini batches
    loss = np.mean(mini_batch_losses)
    losses.append(loss)

Writing model_training/v2.py


In [73]:
%run -i model_training/v2.py

In [74]:
losses

[13.282407255454258,
 11.89312636896253,
 11.658273424521584,
 11.431098318363908,
 11.347354696509584,
 11.260353657416312,
 11.150396601828263,
 11.12010001062907,
 11.028354788002492,
 10.957173211196253,
 10.913551287457512,
 10.86627813810792,
 10.85276506043888,
 10.76486308460306,
 10.769381363101552,
 10.684876007259552,
 10.64103591811613,
 10.651003150306504,
 10.570721113857747,
 10.579957814058254,
 10.52525898939569,
 10.53196040132389,
 10.480088996359344,
 10.474971118272451,
 10.437485200628583,
 10.413304038505272,
 10.369551836520543,
 10.29810795212144,
 10.296200768182198,
 10.284556536480949,
 10.24843507552059,
 10.234301273585245,
 10.21682956632213,
 10.200826351493047,
 10.17850613699628,
 10.13094442902456,
 10.14470091719469,
 10.139005098395682,
 10.096764054597523,
 10.100055080322322,
 10.050454282672643,
 10.048748749472558,
 10.011304151615974,
 9.99980534653822,
 9.965119869506667,
 9.94447472772915,
 9.951626711314015,
 9.927157021844518,
 9.8966456882

In [75]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7534,  0.9320, 23.0112,  0.6130,  0.1556,  0.3211]])), ('0.bias', tensor([0.0732]))])


In [76]:
# Helper Function #2

def mini_batch(device, data_loader, step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        # Send each mini batch to device
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    
    loss = np.mean(mini_batch_losses)
    return loss

In [77]:
%%writefile model_training/v3.py

# Define epochs
n_epochs = 200

losses = []

for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

Writing model_training/v3.py


In [80]:
%run -i data_preparation/v1.py
%run -i model_configuration/v1.py
%run -i model_training/v3.py

In [82]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7268,  1.0011, 20.4338,  0.5707,  0.6372,  0.3646]])), ('0.bias', tensor([0.0553]))])


In [85]:
# Now we're going to add the entire dataset into a TensorDataset, and then
# perform the training-test splits using random_split

In [86]:
%%writefile data_preparation/v2.py

torch.manual_seed(0)

# Build tensors BEFORE split
X_tensor = torch.as_tensor(X).float()
y_tensor = torch.as_tensor(y).float()

# Build dataset with ALL points
dataset = TensorDataset(X_tensor, y_tensor)

# Perform the split
ratio = 0.8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train, n_val])

# Builds a loader of each set
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)
val_loader = DataLoader(
    dataset=val_data,
    batch_size=16,
    shuffle=True,
)

Overwriting data_preparation/v2.py


In [87]:
%run -i data_preparation/v2.py

In [89]:
# Evaluate the model next

# Helper function #3

def make_val_step_fn(model, loss_fn):
    # Builds the function that performs a step in the validation
    # loop
    def perform_val_step_fn(X, y):
        # Step 1 - Computes the predicted output
        yhat = model(X)
        # Step 2 - Compute the loss
        loss = loss_fn(yhat, y)
        # No need to compute step 3 and 4 since we don't
        # update the parameters
        return loss.item()
    
    return perform_val_step_fn

In [92]:
%%writefile model_configuration/v2.py

device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the learning rate
lr = 0.001

torch.manual_seed(0)

# Now we create a model and send it at once to the device
model = nn.Sequential(nn.Linear(6,1)).to(device)

# Define an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define the MSE loss function
loss_fn = nn.MSELoss(reduction="mean")

# Create the train_step function for our model, loss, and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Create the val_step function for our model, loss
val_step_fn = make_val_step_fn(model, loss_fn)

Overwriting model_configuration/v2.py


In [93]:
%run -i model_configuration/v2.py

In [94]:
%%writefile model_training/v4.py

# This time we also include the validatiion step

# Define number of epochs
n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # VALIDATION - no gradients because its in validation!
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)

Writing model_training/v4.py


In [95]:
%run -i model_training/v4.py

In [96]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7365,  0.9475, 21.3789,  0.5911,  0.4888,  0.3591]])), ('0.bias', tensor([0.0197]))])


In [99]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [100]:
%tensorboard --logdir runs

In [101]:
# If we do not specify "test" or any other name, TensorBoard will
# default to runs/CURRENT_DATETIME_HOSTNAME

writer = SummaryWriter("runs/test")

In [104]:
dummy_x, dummy_y = next(iter(train_loader))

# Since our model was sent to device, we need to do the same with the data.
writer.add_graph(model, dummy_x.to(device))

In [105]:
# Now send loss values to TensorBoard using add_scalars

writer.add_scalars(
    main_tag="loss",
    tag_scalar_dict={
        "training": loss,
        "validation": val_loss
    },
    global_step=epoch
)

In [106]:
# Rework the model config to incorporate TensorBoard

In [107]:
%run -i data_preparation/v2.py

In [108]:
%%writefile model_configuration/v3.py

device = "cuda" if torch.cuda.is_available() else "cpu"

# Set learning rate
lr = 0.001

torch.manual_seed(0)

model = nn.Sequential(nn.Linear(6,1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction="mean")

# Create train_step
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Create val step
val_step_fn = make_val_step_fn(model, loss_fn)

# Create a Summary Writer to interface with TensorBoard
writer = SummaryWriter("runs/simple_linear_regression")

# Fetches a single mini-batch so we can use add_graph
x_dummy, y_dummy = next(iter(train_loader))
writer.add_graph(model, x_dummy.to(device))

Writing model_configuration/v3.py


In [109]:
%run -i model_configuration/v3.py

In [110]:
%%writefile model_training/v5.py

# Define the number of epochs
n_epochs = 200

# Define the learning rate
lr = 0.001

losses = []
val_losses = []

for epoch in range(n_epochs):
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)

    # Records both losses for each epoch under tag "loss"
    writer.add_scalars(
        main_tag="loss",
        tag_scalar_dict={
            "training": loss,
            "validation": val_loss
        },
        global_step=epoch
    )

# Close the writer
writer.close()

Writing model_training/v5.py


In [111]:
%run -i model_training/v5.py

In [112]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7664,  0.9821, 21.3793,  0.5757,  0.5077,  0.3798]])), ('0.bias', tensor([0.0416]))])


In [115]:
# Saving and loading models, it is important to know how to checkpoint
# and save our model

# To save our model, we have to save its state (model.state_dict(),
# optimizer.state_dict()), losses, epoch, anything else we need restored

In [116]:
checkpoint = {
    "epoch": n_epochs,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "loss": losses,
    "val_loss": val_losses,
}

torch.save(checkpoint, "model_checkpoint.pth")

In [117]:
# The procedure is the same, whether you are checkpointing a partially
# trained model or saving a fully trained model to deploy

In [118]:
# Resuming training

In [119]:
%run -i data_preparation/v2.py
%run -i model_configuration/v3.py

In [120]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[-0.0031,  0.2190, -0.3360, -0.3004, -0.1572,  0.1095]])), ('0.bias', tensor([-0.0081]))])


In [121]:
# Now we're ready to load the model back

checkpoint = torch.load("model_checkpoint.pth")

model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
saved_epoch = checkpoint["epoch"]
saved_losses = checkpoint["loss"]
saved_val_losses = checkpoint["val_loss"]

model.train() # always use TRAIN for resuming training

Sequential(
  (0): Linear(in_features=6, out_features=1, bias=True)
)

In [122]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7664,  0.9821, 21.3793,  0.5757,  0.5077,  0.3798]])), ('0.bias', tensor([0.0416]))])


In [123]:
%run -i model_training/v5.py

In [124]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7668,  0.9498, 23.1986,  0.5866,  0.1708,  0.3822]])), ('0.bias', tensor([0.0546]))])


In [125]:
losses

[9.002713516855117,
 9.015694255360645,
 9.032064080777403,
 9.002891363715632,
 9.015014864908633,
 8.993514223000185,
 8.996778886370572,
 9.024603309557419,
 9.01635081777277,
 8.980114790512301,
 8.98494594218811,
 9.001210643069996,
 9.0012939201923,
 9.00637609513539,
 9.000493965407674,
 9.015864338023102,
 9.006960377944225,
 9.00266551509384,
 8.994626645662988,
 9.010263282507273,
 9.022635608680488,
 9.012047090040621,
 8.999719053660868,
 9.000959707785022,
 8.995504904856054,
 9.010072109850187,
 9.003563592908302,
 9.005608755488728,
 9.011860081160716,
 9.012771334142956,
 9.00289741181588,
 8.99724913113185,
 9.007617351081636,
 8.992004445048881,
 9.013758222801124,
 9.001717307000813,
 8.999141598240657,
 8.999889146757988,
 8.990330104757033,
 8.9851806927713,
 9.015739355861987,
 9.007411543328017,
 9.003004602914633,
 8.992821498280655,
 9.010519378484066,
 8.98585090233682,
 9.003536385467195,
 8.991954147584678,
 8.988353459255947,
 9.002747203952582,
 8.99566002

In [126]:
# Loss is not decreasing so we have a fully trained model

In [127]:
%run -i model_configuration/v3.py

In [128]:
# checkpoint = torch.load("model_checkpoint.pth")
# model.load_state_dict(checkpoint["model_state_dict"])
# print(model.state_dict())

OrderedDict([('0.weight', tensor([[ 0.7664,  0.9821, 21.3793,  0.5757,  0.5077,  0.3798]])), ('0.bias', tensor([0.0416]))])


In [129]:
# new_inputs = torch.tensor([[.20], [.34], [.57]])

In [ ]:
# The data prep, model config, and model training we did is the
# general structure you'll use over and over again for training PyTorch
# models